In [3]:
import pandas as pd
import numpy as np
import os
import kaggle
import ast

In [4]:
#from kaggle.api.kaggle_api_extended import KaggleApi
#api = KaggleApi()
#api.authenticate()

#api.dataset_download_files('shuyangli94/food-com-recipes-and-user-interactions')

In [5]:
os.chdir(r"D:\recipe_recommender\food-com-recipes-and-user-interactions")

In [6]:
interactions_train = pd.read_csv("interactions_train.csv")
pp_recipes = pd.read_csv("PP_recipes.csv")
pp_users = pd.read_csv("PP_users.csv")
raw_interactions = pd.read_csv("RAW_interactions.csv")
raw_recipes = pd.read_csv("RAW_recipes.csv")

In [7]:
interactions_train.head()

,user_id,recipe_id,date,rating,u,i
0,2046,4684,2000-02-25,5.0,22095,44367
1,2046,517,2000-02-25,5.0,22095,87844
2,1773,7435,2000-03-13,5.0,24732,138181
3,1773,278,2000-03-13,4.0,24732,93054
4,2046,3431,2000-04-07,5.0,22095,101723


In [8]:
pp_recipes.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [9]:
id_dict = {}

for i in range(len(pp_recipes)):
    id_dict[pp_recipes['i'][i]] = pp_recipes['id'][i]


In [10]:
pp_recipes.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [19]:
raw_recipes[raw_recipes['id'] == 137739]['ingredients']

0    ['winter squash', 'mexican seasoning', 'mixed ...
Name: ingredients, dtype: object

Turn each item into a row making item (the recipe) the target variable. The predictor variables will be n_steps, n_ingredients, ingredient_tokens, nutrition, minutes. A user will then fll out a form specifying the predictor variables and based on the predictor variables a model will predict what recipe they may like. 

In [58]:
new_df = pd.DataFrame()
recipes, ratings = [], []

for i in range(len(pp_users)):
    item = ast.literal_eval(pp_users['items'][i])
    rating = ast.literal_eval(pp_users['ratings'][i])

    for j in range(len(item)):
        recipes.append(id_dict[item[j]])
        ratings.append(rating[j])

new_df['recipe'] = recipes
new_df['rating'] = ratings

In [59]:
new_df2 = new_df.groupby(['recipe']).mean()

In [60]:
new_df2 = new_df2.reset_index(drop=False)

In [61]:
new_df2 = new_df2[new_df2['rating'] > 3]

In [62]:
raw_recipes2 = raw_recipes[raw_recipes['id'].isin(list(new_df['recipe']))]

In [63]:
raw_recipes2 = raw_recipes2.reset_index(drop=True)

In [67]:
new_df2 = new_df2.reset_index(drop=True)

The predictor variables will be n_steps, n_ingredients, ingredient_tokens, nutrition, minutes

In [70]:
n_steps, n_ingredients, nutrition, minutes, ingreds = {},{},{},{},{}

for i in range(len(raw_recipes2)):
    recipe = raw_recipes2['id'][i]
    steps = raw_recipes2['steps'][i]
    ingredients = ast.literal_eval(raw_recipes2['ingredients'][i])
    nutr = raw_recipes2['nutrition'][i]
    mins = raw_recipes2['minutes'][i]

    n_steps[recipe] = steps
    n_ingredients[recipe] = len(ingredients)
    nutrition[recipe] = nutr
    minutes[recipe] = mins
    ingreds[recipe] = ingredients

n_steps2, n_ingredients2, nutrition2, minutes2, ingreds2 = [],[],[],[],[]

for j in range(len(new_df2)):
    recipe = new_df2['recipe'][j]

    n_steps2.append(n_steps[recipe])
    n_ingredients2.append(n_ingredients[recipe])
    nutrition2.append(nutrition[recipe])
    minutes2.append(minutes[recipe])
    ingreds2.append(ingreds[recipe])


new_df2['n_steps'] = n_steps2
new_df2['n_ingredients'] = n_ingredients2
new_df2['nutrition'] = nutrition2
new_df2['minutes'] = minutes2
new_df2['ingredients'] = ingreds2


In [71]:
new_df2.head()

,recipe,rating,n_steps,n_ingredients,nutrition,minutes,ingredients
0,38,4.333333,"['toss 2 cups berries with sugar', 'let stand ...",4,"[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",1485,"[blueberries, granulated sugar, vanilla yogurt..."
1,40,4.800000,"['into a 1 quart jar with tight fitting lid , ...",6,"[311.1, 0.0, 308.0, 0.0, 0.0, 0.0, 27.0]",35,"[sugar, lemons, rind of, fresh water, fresh le..."
2,45,4.000000,"['preheat oven to 350f', 'make pie crust , usi...",9,"[228.0, 10.0, 98.0, 11.0, 8.0, 8.0, 12.0]",80,"[sugar, margarine, egg, egg whites, flour, sal..."
3,46,5.000000,"['slice the cucumber in four lengthwise , then...",6,"[4.3, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",25,"[rice vinegar, thangkwa, hom daeng, nam som pa..."
4,49,4.000000,['cook mushrooms in 2 tbsp butter in a large s...,10,"[627.7, 38.0, 8.0, 35.0, 115.0, 64.0, 4.0]",75,"[fresh mushrooms, butter, boneless skinless ch..."


In [73]:
raw_recipes2[raw_recipes2['id'] == 38]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
88343,low fat berry blue frozen dessert,38,1485,1533,1999-08-09,"['weeknight', 'time-to-make', 'course', 'prepa...","[170.9, 3.0, 120.0, 1.0, 6.0, 6.0, 12.0]",13,"['toss 2 cups berries with sugar', 'let stand ...","this is yummy and low-fat, it always turns out...","['blueberries', 'granulated sugar', 'vanilla y...",4
